# 📚 Personal AI Tutor  
**Anurag Rai**



## 🧠 Problem Statement

Students often struggle to revise large textbooks or lecture materials efficiently. Traditional methods of studying are time-consuming, lack interactivity, and offer no personalized assistance. There's a need for a smarter, AI-driven tutor that can help learners quickly understand, review, and test their knowledge based on their own study material.



## ✅ Solution

This project builds a **Personal AI Tutor** using **Google’s Gemini 1.5 Flash model**. The tutor reads a textbook or lecture note PDF, processes it, and provides:



### 1. Contextual Question Answering  
- Users can ask questions in natural language.  
- The system uses **Retrieval-Augmented Generation (RAG)** to fetch relevant content and generate accurate, grounded answers.



### 2. Quiz Generation  
- Automatically creates **multiple-choice questions** based on the material.  
- Output is structured in **JSON format**, making it usable for quizzes or further evaluation.



### 3. PDF Understanding & Embedding Search  
- The system processes **unstructured documents**, converts them into embeddings, and stores them using **FAISS** for fast semantic retrieval.



## 🧪 Gen AI Capabilities Demonstrated

| Capability                        | How It’s Used                                           |
|----------------------------------|---------------------------------------------------------|
| **Document Understanding**       | Parses and chunks PDF content                          |
| **Embeddings & Vector Search**   | Encodes content and retrieves relevant info             |
| **RAG (Retrieval-Augmented Gen)**| Combines search and generation for QA                   |
| **Few-shot Prompting**           | Guides the model in generating quiz questions           |
| **Structured Output (JSON Mode)**| Generates quizzes in machine-readable format            |


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 🛠️ Setup:
Installing Required Libraries 

In [2]:
!pip install --quiet PyMuPDF langchain faiss-cpu tqdm google-generativeai

## 📄 Upload and Extract Text from PDF
This code extracts the raw text from a PDF file using the **PyMuPDF** (`fitz`) library. It reads all pages of the provided PDF and concatenates the extracted text for further processing.


In [3]:
import fitz  # PyMuPDF
from pathlib import Path

def extract_text_from_pdf(pdf_path, layout_mode="text"):
    try:
        doc = fitz.open(pdf_path)
        all_text = []

        for page_num, page in enumerate(doc):
            if layout_mode == "blocks":
                text = page.get_text("blocks")
                page_text = "\n".join(block[4] for block in text if block[4].strip())
            else:
                page_text = page.get_text("text", sort=True)

            all_text.append(page_text.strip())

        doc.close()
        return "\n\n".join(all_text)

    except Exception as e:
        print(f"❌ Error reading PDF: {e}")
        return None

# Provide your PDF path here
pdf_path = "data/Agile unit 1.pdf"
raw_text = extract_text_from_pdf(pdf_path)

if raw_text:
    print("✅ PDF loaded successfully!")
    print(f"📄 Total characters extracted: {len(raw_text)}")
    print("🔍 Preview of extracted text:\n")
    print("\n".join(raw_text.splitlines()[:20]))  # Show first 20 lines
else:
    print("❌ Failed to load PDF.")


✅ PDF loaded successfully!
📄 Total characters extracted: 29023
🔍 Preview of extracted text:

ARTIFICIAL INTELLIGENCE IN AGILE
          SYSTEMS

SYLLABUS


                    UNIT -  I
Introduction to AI techniques, Intelligent Agents, Problem
Solving with AI, Intelligent Agents, Structure of Agents,
Agile Alliance, Principles of Agile Practices, Practices of
Extreme  Programming,  Planning   –Initial  Exploration,
Release Planning   ,  Iteration planning   , Task  planning,
Challenges in Traditional Systems, Real time applications of
AI in Agile systems, Test driven development, Acceptance
Test,    Serendipitious    Architecture,    Serendipitious
decoupling






## ✂️ Chunk the Text
This code uses **LangChain's RecursiveCharacterTextSplitter** to split the raw text into smaller chunks of 500 characters, with an overlap of 50 characters between chunks. This helps in processing large text for tasks like embeddings and semantic search.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  
)

# Split the raw text into chunks
chunks = text_splitter.split_text(raw_text)
print(f"Number of Chunks: {len(chunks)}")
print(chunks[0])

Number of Chunks: 37
ARTIFICIAL INTELLIGENCE IN AGILE
          SYSTEMS

SYLLABUS


                    UNIT -  I
Introduction to AI techniques, Intelligent Agents, Problem
Solving with AI, Intelligent Agents, Structure of Agents,
Agile Alliance, Principles of Agile Practices, Practices of
Extreme  Programming,  Planning   –Initial  Exploration,
Release Planning   ,  Iteration planning   , Task  planning,
Challenges in Traditional Systems, Real time applications of
AI in Agile systems, Test driven development, Acceptance
Test,    Serendipitious    Architecture,    Serendipitious
decoupling





                                                                                                        2

SYLLABUS


## 💾 Wrap Chunks with Metadata (for future use)
This code converts the text chunks into **LangChain Document** objects, where each chunk is associated with metadata (e.g., a unique identifier). This structure helps organize the text and its metadata for further processing, like embedding or querying.


In [5]:
from langchain.schema import Document

docs = [Document(page_content=chunk, metadata={"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]


## 🔐 Set Up Gemini Embeddings

We install the `google-generativeai` SDK and securely load the Gemini API key using **Kaggle Secrets**.  
This key is used to configure the Gemini client, enabling us to access the embedding model for downstream tasks such as semantic search and retrieval.


In [6]:
# 📦 Install Google Generative AI SDK
!pip install --quiet google-generativeai

# 🔐 Securely retrieve API key from Kaggle Secrets
import os
from dotenv import load_dotenv
import google.generativeai as genai
from tqdm import tqdm

# 🔐 Securely load API key from .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Check if the API key is loaded
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it is set in your .env file.")

# 🔧 Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)

# 🔁 Embedding Function
def get_embedding(text):
    try:
        response = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type="retrieval_document"
        )
        return response["embedding"]
    except Exception as e:
        print(f"[ERROR] Embedding failed: {e}")
        return None


## 🧠 Generate Embeddings for Chunks

We use the Gemini embedding model (`embedding-001`) to convert each chunk of text into a numerical vector.  
These embeddings capture the semantic meaning of the content and will be used for efficient similarity search and retrieval.


In [7]:
import numpy as np

embeddings = []
valid_docs = []

for doc in tqdm(docs, desc="🔍 Generating embeddings"):
    embedding = get_embedding(doc.page_content)
    
    if embedding:
        embeddings.append(embedding)
        valid_docs.append(doc)  # Store only successfully embedded docs
    else:
        print("⚠️ Skipped a doc due to embedding error.")

# 🧠 Replace docs with valid ones (to stay in sync with embeddings)
docs = valid_docs
print(f"✅ Generated {len(embeddings)} embeddings (out of {len(chunks)} chunks)")


🔍 Generating embeddings: 100%|██████████| 37/37 [00:33<00:00,  1.11it/s]

✅ Generated 37 embeddings (out of 37 chunks)


## 📦 Store in FAISS Vector Store

We use FAISS to store the embeddings of our text chunks.  
This helps us quickly find similar content later when answering questions.


In [8]:
import faiss

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


## 📊 System Metrics
This cell calculates and displays important system metrics after the FAISS index creation.  
It outputs:
- The number of pages processed from the PDF.
- The number of text chunks created from the document.
- The embedding dimension used.
- The size of the FAISS index, indicating how many vectors are stored.

These metrics provide insights into the document processing pipeline and FAISS indexing process.

In [9]:
page_count = len(raw_text.split('\f'))  

print("\n=== System Metrics ===")
print(f"- PDF Pages Processed: {page_count}")  
print(f"- Text Chunks: {len(chunks)}")
print(f"- Embedding Dimension: {dimension}")
print(f"- FAISS Index Size: {index.ntotal} vectors")


=== System Metrics ===
- PDF Pages Processed: 1
- Text Chunks: 37
- Embedding Dimension: 768
- FAISS Index Size: 37 vectors


## 🔍 Basic Semantic Search Demo

In [10]:
def search(query, k=5, filter_keywords=None, show_scores=True):
    query_embedding = get_embedding(query)
    if query_embedding is None:
        print("❌ Failed to embed query.")
        return []

    distances, indices = index.search(np.array([query_embedding]), k)
    results = []
    
    for idx, score in zip(indices[0], distances[0]):
        content = docs[idx].page_content
        
        # Apply optional keyword filtering
        if filter_keywords:
            if not any(keyword.lower() in content.lower() for keyword in filter_keywords):
                continue
        
        results.append((content, score))
    
    # Show results
    for i, (content, score) in enumerate(results[:3]):  # Show top 3
        print(f"\n🔹 Result {i+1} (Similarity Score: {score:.2f}):\n")
        print(content[:700].strip() + ("\n..." if len(content) > 700 else ""))  # Truncate long outputs
    
    return results[:3]

# Example test
_ = search("What is reinforcement learning?", filter_keywords=["reinforcement", "agent", "reward", "policy"])


🔹 Result 1 (Similarity Score: 0.58):

4/13/2025                                                                                     66

Planning vs. Problem solving
 • Planning agent is very similar to problem solving agent
   – Constructs plans to achieve goals, then executes them
 • Planning    is  more   powerful  because   of   the
   representations and methods used
 • Search - proceeds through plan space rather than state
   space
 •  Sub-goals  - planned  independently,   it  reduce  the
   complexity of the planning problem





4/13/2025                                                                                     67

Planning Agents

🔹 Result 2 (Similarity Score: 0.62):

Artificial Neural Networks

Markov Decision Process

• A Markov Decision Process (MDP) is a framework
  for decision-making modeling where in some
  situations the outcome is partly random and
  partly based on the input of the decision maker.
• Another application where MDP is used is
  optimized plan

## 🔄 Define RAG Pipeline

We define a Retrieval-Augmented Generation (RAG) pipeline that combines relevant text chunks with a question to generate an accurate, context-based answer.  
The pipeline retrieves the top documents and uses them to guide the model’s response.


In [11]:
def rag_answer(query, k=4, temperature=0.3, filter_keywords=None):
    try:
        # 🔍 Retrieve relevant documents using the improved search
        search_results = search(query, k=k + 2, filter_keywords=filter_keywords, show_scores=False)
        if not search_results:
            return "I couldn't find relevant content in the PDF to answer that question."

        # 📚 Prepare context for the prompt
        context = "\n\n".join([
            f"Document {i+1}:\n{chunk.strip()}" 
            for i, (chunk, _) in enumerate(search_results[:k])
        ])

        # 🧠 Prompt engineering
        prompt = f"""You are a helpful tutor. Based on the context below, answer the question.
If the answer isn't in the context, say you don't know but make an educated guess.

Context:
{context}

Question:
{query}

Provide a concise answer with 1–2 examples if applicable:"""

        # 🎯 Generate the answer using Gemini
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        generation_config = genai.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            top_k=40
        )
        response = model.generate_content(prompt, generation_config=generation_config)

        return response.text if hasattr(response, 'text') else response.parts[0].text

    except Exception as e:
        print(f"Error generating answer: {e}")
        return "I encountered an error while processing your question. Please try again."

## 💬 Testing It

In [12]:
question = "What is test-driven development in agile?"
print(rag_answer(question, filter_keywords=["test", "TDD", "agile", "development"]))


🔹 Result 1 (Similarity Score: 0.36):

Test driven development

• Test Driven Development (TDD) is a software
  development practice that focuses on creating
  unit test cases before developing the actual
  code.
•  It  is an  iterative approach  that combines
  programming, the creation of unit tests, and
  refactoring

Test driven development

• TDD in agile is a framework that emphasizes
  the creation of unit test cases prior to writing
  the real code.
•  It  is an  iterative process that incorporates
  programming, unit testing, and refactoring.
•  It   is  inevitable   for  mistakes  to  occur
  throughout the design and coding phases of
  software development

Acceptance Test

• Acceptance  Test  -Driven  Development,  or
...

🔹 Result 2 (Similarity Score: 0.51):

Early and continuous delivery of
          valuable software

•  Agile aims to deliver a functioning product in the very
   first development iteration. It will be a long way from
  being finished; but  it just has to

## 🔄 Implementation of RAG (Retrieval Step)
This cell defines the `search()` function, which is responsible for retrieving the most relevant chunks of text from your PDF or document based on a user query.

- It uses vector similarity to find the best-matching passages.
- Returns the top `k` relevant chunks (default is 3).

In [13]:
!pip install --quiet rapidfuzz

from rapidfuzz import fuzz
import time

def evaluate_rag(questions, expected_answers):
    """
    Evaluate RAG performance on sample questions
    Returns accuracy and average response time
    """
    results = []
    start_time = time.time()
    
    for q, expected in zip(questions, expected_answers):
        start_q = time.time()
        answer = rag_answer(q)
        end_q = time.time()
        
        # Fuzzy matching for semantic similarity
        similarity_score = fuzz.partial_ratio(expected.lower(), answer.lower())
        
        # If similarity score > threshold (say 55), consider the answer correct
        correct = similarity_score > 55
        
        results.append({
            "question": q,
            "answer": answer,
            "expected": expected,
            "similarity_score": similarity_score,
            "correct": correct,
            "time": end_q - start_q
        })
    
    total_time = time.time() - start_time
    accuracy = sum(r['correct'] for r in results) / len(results)
    
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Average response time: {total_time/len(questions):.2f}s")
    return results

# Example usage with two questions:
sample_questions = [
    "What is test-driven development in agile?",
    "Explain the concept of a planning agent in AI."
]

sample_answers = [
    "TDD is an iterative software development process where tests are written before the code.",
    "A planning agent constructs and executes plans to reach goals."
]

evaluation_results = evaluate_rag(sample_questions, sample_answers)

# To inspect the results
for result in evaluation_results:
    print(f"\nQuestion: {result['question']}")
    print(f"Answer: {result['answer']}")
    print(f"Expected: {result['expected']}")
    print(f"Similarity Score: {result['similarity_score']}")
    print(f"Correct: {result['correct']}")
    print(f"Time: {result['time']:.2f}s")


🔹 Result 1 (Similarity Score: 0.36):

Test driven development

• Test Driven Development (TDD) is a software
  development practice that focuses on creating
  unit test cases before developing the actual
  code.
•  It  is an  iterative approach  that combines
  programming, the creation of unit tests, and
  refactoring

Test driven development

• TDD in agile is a framework that emphasizes
  the creation of unit test cases prior to writing
  the real code.
•  It  is an  iterative process that incorporates
  programming, unit testing, and refactoring.
•  It   is  inevitable   for  mistakes  to  occur
  throughout the design and coding phases of
  software development

Acceptance Test

• Acceptance  Test  -Driven  Development,  or
...

🔹 Result 2 (Similarity Score: 0.51):

Early and continuous delivery of
          valuable software

•  Agile aims to deliver a functioning product in the very
   first development iteration. It will be a long way from
  being finished; but  it just has to

## 🧠 Generate Quiz Prompt 

This step builds a structured prompt using context and sends it to a language model to generate multiple-choice questions.  
It uses **few-shot prompting** by providing an example JSON format for the model to follow.  
The model returns questions with options, the correct answer, difficulty, and explanations.

In [14]:
import json
import re
import google.generativeai as genai

def build_quiz_prompt(context: str, num_questions: int = 3) -> str:
    return f"""
You are an AI tutor. Based on the context below, create {num_questions} multiple-choice questions.

Each question must have:
- A clear, standalone question text
- 4 labeled options (A, B, C, D) in dictionary format
- Only 1 correct answer
- Difficulty level: "easy", "medium", or "hard"
- A brief explanation for the correct answer

Context:
\"\"\"
{context}
\"\"\"

Respond with valid JSON in the following format:
{{
  "questions": [
    {{
      "question": "What is AI?",
      "options": {{
        "A": "A programming language",
        "B": "A type of algorithm",
        "C": "Simulation of human intelligence",
        "D": "A computer virus"
      }},
      "correct_answer": "C",
      "difficulty": "easy",
      "explanation": "AI refers to the simulation of human intelligence by machines."
    }}
  ]
}}
""".strip()


def clean_response_to_json(text: str) -> dict:
    """Attempts to clean and parse a Gemini response into JSON."""
    try:
        # Remove markdown-style code blocks
        cleaned = re.sub(r"^```(?:json)?|```$", "", text, flags=re.MULTILINE).strip()
        
        # Normalize quotes (optional)
        cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

        return json.loads(cleaned)

    except json.JSONDecodeError as e:
        print("[JSONDecodeError]", e)
        print("Problematic response snippet:", text[:300], "...")
        return {"questions": []}


def generate_quiz_from_prompt(quiz_prompt: str) -> dict:
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        response = model.generate_content(
            quiz_prompt,
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                temperature=0.7
            )
        )

        # Extract response text
        raw_text = getattr(response, 'text', None) or getattr(response.parts[0], 'text', '')
        quiz_json = clean_response_to_json(raw_text)

        # Structural validation
        if "questions" not in quiz_json or not isinstance(quiz_json["questions"], list):
            raise ValueError("JSON is missing a valid 'questions' field.")

        return quiz_json

    except Exception as e:
        print("[ERROR] Quiz generation failed:", str(e))
        return {"questions": []}

## 📝 Generate Quiz 

This cell takes a retrieved context chunk, builds a quiz prompt, and generates a quiz using the Gemini model.  
It parses and prints the questions in readable JSON format.  
This step turns study content into interactive quiz form for active learning.

In [15]:
# Sample usage
sample_context = search("Test-driven development")[0] 
quiz_prompt = build_quiz_prompt(sample_context, num_questions=3)
quiz = generate_quiz_from_prompt(quiz_prompt)

# Pretty-print the generated quiz
if quiz["questions"]:
    print("✅ Quiz Generated Successfully:\n")
    print(json.dumps(quiz, indent=2, ensure_ascii=False))
else:
    print("⚠️ Quiz generation failed or returned no questions.")


🔹 Result 1 (Similarity Score: 0.40):

Test driven development

• Test Driven Development (TDD) is a software
  development practice that focuses on creating
  unit test cases before developing the actual
  code.
•  It  is an  iterative approach  that combines
  programming, the creation of unit tests, and
  refactoring

Test driven development

• TDD in agile is a framework that emphasizes
  the creation of unit test cases prior to writing
  the real code.
•  It  is an  iterative process that incorporates
  programming, unit testing, and refactoring.
•  It   is  inevitable   for  mistakes  to  occur
  throughout the design and coding phases of
  software development

Acceptance Test

• Acceptance  Test  -Driven  Development,  or
...

🔹 Result 2 (Similarity Score: 0.51):

4/13/2025                                                                                     68

Release Planning

• Agile release planning is a valuable technique
  for building customer-centric products and
  proje

### ✅ Conclusion
This project, created as part of the **GenAI Intensive Course Capstone 2025Q1**, presents a working and effective use of Generative AI: a **Personal AI Tutor** that takes static learning content and turns it into an interactive learning assistant.

Through the use of **Google Gemini 1.5 Flash**, **Retrieval-Augmented Generation (RAG)**, and **FAISS-based semantic search**, this solution exhibits some of the main GenAI capabilities:

- **Document understanding**: Parsing and chunking educational PDFs

- **Search-based embedding**: Returning contextually relevant content
- **QA in natural language**: Responding to user queries with grounded, accurate answers
- **Quiz generation**: Generating machine-readable, structured multiple-choice questions
The work demonstrates an excellent grasp of GenAI workflows, prompt engineering, and real-world use case design.
This AI mentor not only facilitates self-directed learning but also illustrates how **GenAI** can tailor education at scale.

